In [ ]:
import numpy as np 
import pandas as pd 
import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

In [ ]:
labels=pd.read_csv("../input/traffic-signs-classification/labels.csv")
labels.head(45)

In [ ]:
dic={}
for i in range(labels.shape[0]):
    dic[i]=labels.iloc[i]['Name']

In [ ]:
print(dic)

In [ ]:
labels[labels['Name']=='Turn right ahead'].index[0]

In [ ]:
from PIL import Image
im = Image.open('../input/traffic-signs-classification/myData/0/00000_00000.jpg')
im

In [ ]:
im.size

In [ ]:
from keras.preprocessing import image
x=[]
y=[]
for dirs in os.listdir('../input/traffic-signs-classification/myData'):
    for files in os.listdir("../input/traffic-signs-classification/myData/"+dirs):
        sign_img = np.array(image.load_img("../input/traffic-signs-classification/myData/"+dirs+"/"+files,target_size = (32,32)))
        x.append(sign_img)
        y.append(int(dirs))

In [ ]:
x[0:1]

In [ ]:
y[0]

In [ ]:
from sklearn.utils import shuffle
x_shuffled, y_shuffled = shuffle(x, y)


In [ ]:
def plot_image(x,y, index):
    image = plt.imshow(x[index])
    l=plt.title(dic[y[index]])
    ax.grid(False)
    ax.axis('off')    
    return image, l
fig = plt.figure(figsize=(20, 20))

for i in range(20):
    ax = fig.add_subplot(5, 4, i + 1)
    plot_image(x_shuffled,y_shuffled, i)

plt.show()

In [ ]:
x_shuffled[0].shape

In [ ]:
len(y_shuffled)

In [ ]:
import keras
y_categorical = keras.utils.to_categorical(y_shuffled, 43)

In [ ]:
from sklearn.model_selection import train_test_split
x_train,x_rest,y_train,y_rest=train_test_split(x_shuffled,y_categorical,test_size=0.1)

In [ ]:
x_test,x_val,y_test,y_val=train_test_split(x_rest,y_rest,test_size=0.5)

In [ ]:
len(x_train),len(y_train)

In [ ]:
len(x_test),len(y_test)

In [ ]:
len(x_val),len(y_val)

In [ ]:
x_train[0]

In [ ]:
y_train[0]

In [ ]:
from tensorflow.keras.applications.resnet import ResNet50
from tensorflow.keras.applications.vgg16 import VGG16
from keras import backend as k
from keras.preprocessing.image import ImageDataGenerator,load_img, img_to_array
from keras.models import Sequential, Model
from keras.layers import Conv2D, MaxPooling2D,GlobalAveragePooling2D
from keras.layers import Activation, Dropout, BatchNormalization, Flatten, Dense, AvgPool2D,MaxPool2D
from keras.models import Sequential, Model
from keras.optimizers import Adam, SGD, RMSprop
import keras
import tensorflow as tf

In [ ]:
vgg_model = VGG16(weights= 'imagenet', include_top=False, input_shape= (32,32,3))

In [ ]:
model=keras.Sequential()
for layer in vgg_model.layers:
    model.add(layer)
for layer in model.layers:
    layer.trainable=False
model.add(Flatten())
model.add(Dropout(0.4))
model.add(Dense(256,activation='relu'))
model.add(Dense(43,activation='softmax'))
model.summary()

In [ ]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
from keras.callbacks import EarlyStopping,ModelCheckpoint

stop = EarlyStopping(
    monitor='val_accuracy', 
    mode='max',
    patience=6
)

checkpoint= ModelCheckpoint(
    filepath='./',
    save_weights_only=True,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True)

In [ ]:
x_train_array=np.array(x_train)
y_train_array=np.array(y_train)
x_train_scaled=x_train_array/255.0

In [ ]:
x_train_scaled[1]

In [ ]:
y_train[0]

In [ ]:
x_val_array=np.array(x_val)
y_val_array=np.array(y_val)
x_val_scaled=x_val_array/1.0
x_val_scaled=x_val_scaled/255


In [ ]:
x_val_scaled[0]

In [ ]:
y_train_array[0].shape

In [ ]:
# history =  model.fit(x_train_scaled, y_train_array,validation_data = (x_val_scaled, y_val_array), batch_size =256, epochs =50, verbose = 1, callbacks = [stop, checkpoint])

In [ ]:
model.save("Vgg_traffic_model.h5")

In [ ]:
model=keras.models.load_model("../input/vgg-traffic-sign-model/Vgg_traffic_model.h5")

In [ ]:
x_test_array=np.array(x_test)
x_test_scaled=x_test_array/255.0

In [ ]:
x_test_scaled[0]

In [ ]:
y_test_array=np.array(y_test)

In [ ]:
loss, acc = model.evaluate(x_test_scaled, y_test_array)
print('Accuracy: ', acc, '\nLoss    : ', loss)

In [ ]:
history=model.history

In [ ]:
# plt.figure(figsize=(12, 5))

# plt.plot(model['accuracy'])
# plt.plot(history.history['val_accuracy'])
# plt.title('model accuracy')
# plt.ylabel('accuracy')
# plt.xlabel('epoch')
# plt.legend(['train', 'validation'], loc='upper left')
# plt.show()
# plt.figure(figsize=(12, 5))

# plt.plot(history.history['loss'])
# plt.plot(history.history['val_loss'])
# plt.title('model loss')
# plt.ylabel('loss')
# plt.xlabel('epoch')
# plt.legend(['train', 'validation'], loc='upper left')
# plt.show()


In [ ]:
pred = np.argmax(model.predict(x_test_scaled), axis = 1)

In [ ]:
pred

In [ ]:
actual=np.argmax(y_test_array, axis = 1)

In [ ]:
actual

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report
labels = [dic[i] for i in range(43)]
print(classification_report(actual, pred, target_names = labels))

In [ ]:
import seaborn as sns
cmat = confusion_matrix(actual, pred)
plt.figure(figsize=(16,16))
sns.heatmap(cmat, annot = True, cbar = False, cmap='YlGnBu', fmt="d", xticklabels=labels, yticklabels=labels);

In [ ]:
y_test[0]

In [ ]:
actual[3]

In [ ]:
pred[2]

In [ ]:
def plot_image(x_test,pred,actual, i,dic):
    image = plt.imshow(x_test[i])
    l=plt.title(dic[pred[i]])
    ax.grid(False)
    ax.axis('off')    
    return image, l
fig = plt.figure(figsize=(20, 20))

for i in range(20):
    ax = fig.add_subplot(5, 4, i + 1)
    plot_image(x_test,pred,actual, i,dic)

plt.show()

In [ ]:
resnet_model = ResNet50(weights= 'imagenet', include_top=False, input_shape= (32,32,3))

In [ ]:
x = resnet_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.5)(x)
predictions = Dense(43, activation= 'softmax')(x)
model2 = Model(inputs = resnet_model.input, outputs = predictions)

In [ ]:
model2.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
from keras.callbacks import EarlyStopping,ModelCheckpoint

stop = EarlyStopping(
    monitor='val_accuracy', 
    mode='max',
    patience=3
)

checkpoint= ModelCheckpoint(
    filepath='./',
    save_weights_only=True,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True)

In [ ]:
history =  model2.fit(x_train_scaled, y_train_array,validation_data = (x_val_scaled, y_val_array), batch_size =256, epochs =50, verbose = 1, callbacks = [stop, checkpoint])

In [ ]:
plt.figure(figsize=(12, 5))

plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()
plt.figure(figsize=(12, 5))

plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()


In [ ]:
pred2 = np.argmax(model2.predict(x_test_scaled), axis = 1)

In [ ]:
actual=np.argmax(y_test_array, axis = 1)

In [ ]:
pred2

In [ ]:
actual

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report
labels = [dic[i] for i in range(43)]
print(classification_report(actual, pred2, target_names = labels))

In [ ]:
loss, acc = model2.evaluate(x_test_scaled, y_test_array)
print('Accuracy: ', acc, '\nLoss    : ', loss)

In [ ]:
import seaborn as sns
cmat = confusion_matrix(actual, pred2)
plt.figure(figsize=(16,16))
sns.heatmap(cmat, annot = True, cbar = False, cmap='YlGnBu', fmt="d", xticklabels=labels, yticklabels=labels);

In [ ]:
def plot_image(x_test,pred,actual, i,dic):
    image = plt.imshow(x_test[i])
    l=plt.title(dic[pred[i]])
    ax.grid(False)
    ax.axis('off')    
    return image, l
fig = plt.figure(figsize=(20, 20))

for i in range(20):
    ax = fig.add_subplot(5, 4, i + 1)
    plot_image(x_test,pred2,actual, i,dic)

plt.show()

In [ ]:
model2.save("resnet_traffic_sign.h5")